In [1]:
#setup imports and make ure the files we care about exist
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os.path

features_path = "/home/jaywalker/MachineLearning/UNSW-NB15-CSV/NUSW-NB15_features.csv"
print("Features file found!" if os.path.isfile("/home/jaywalker/MachineLearning/UNSW-NB15-CSV/NUSW-NB15_features.csv") else "")

Features file found!


In [2]:
#quick peek at the features and cleaning up column names for easier indexing
features_df = pd.read_csv(features_path, encoding="latin-1")
for i in range(len(features_df.columns.values)):
    features_df.columns.values[i] = str(features_df.columns.values[i]).strip().lower()
    
    
print(features_df.columns) #cleaned up column names

#lower case all the types
for i in range(len(features_df)):
    features_df.loc[i, ['type']] = str(features_df['type'][i]).strip().lower()
    features_df.loc[i, ['name']] = str(features_df['name'][i]).strip().lower()

print(features_df[['name', 'type']])

Index(['no.', 'name', 'type', 'description'], dtype='object')
                name       type
0              srcip    nominal
1              sport    integer
2              dstip    nominal
3             dsport    integer
4              proto    nominal
5              state    nominal
6                dur      float
7             sbytes    integer
8             dbytes    integer
9               sttl    integer
10              dttl    integer
11             sloss    integer
12             dloss    integer
13           service    nominal
14             sload      float
15             dload      float
16             spkts    integer
17             dpkts    integer
18              swin    integer
19              dwin    integer
20             stcpb    integer
21             dtcpb    integer
22           smeansz    integer
23           dmeansz    integer
24       trans_depth    integer
25       res_bdy_len    integer
26              sjit      float
27              djit      float
28        

In [3]:
#quick peek at the data
training_set_path = "/home/jaywalker/MachineLearning/UNSW-NB15-CSV/train_test/UNSW_NB15_training-set.csv"
training_df = pd.read_csv(training_set_path, encoding="latin-1")
print(training_df[:1])
#Of COURSE this file is organized differently than the features file describes.
#Why would I expect differently?

   id       dur proto service state  spkts  dpkts  sbytes  dbytes        rate  \
0   1  0.000011   udp       -   INT      2      0     496       0  90909.0902   

   ...    ct_dst_sport_ltm  ct_dst_src_ltm  is_ftp_login  ct_ftp_cmd  \
0  ...                   1               2             0           0   

   ct_flw_http_mthd  ct_src_ltm  ct_srv_dst  is_sm_ips_ports  attack_cat  \
0                 0           1           2                0      Normal   

   label  
0      0  

[1 rows x 45 columns]


In [4]:
#so we'll use a file from the "full" dataset instead
nb15_1_path = "/home/jaywalker/MachineLearning/PacketGAN/UNSW-NB15_1_clean.csv"
packet_data_df = pd.read_csv(nb15_1_path, encoding="latin-1", names=features_df['name'], header=None)
print(packet_data_df.columns)

/home/jaywalker/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'sload', 'dload',
       'spkts', 'dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'sjit', 'djit', 'stime',
       'ltime', 'sintpkt', 'dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'label'],
      dtype='object')


In [5]:
#for each feature of type "nominal" or "integer" count how many classes exist
#print(packet_data_df['label'].unique()) #identify the different values

for label, feature_type in features_df[['name', 'type']].values:
    nunique = packet_data_df[packet_data_df['label'] == 1][label].nunique()
    if nunique < 20:
        value_list = packet_data_df[packet_data_df['label'] == 1][label].unique().tolist()
        print(label + ": " , end='')
        print(value_list, end='')
        print(" type: " + str(feature_type))
    else:
        print(label + ": " + str(nunique) + " type: " + str(feature_type))
        
        
#how many attack packets do we have compared to non-attack packets?
print("Normal packets: ", len(packet_data_df[packet_data_df['label'] == 0].index))
print("Attack packets: ", len(packet_data_df[packet_data_df['label'] == 1].index))

srcip: ['175.45.176.0', '175.45.176.1', '175.45.176.2', '175.45.176.3'] type: nominal
sport: 9607 type: integer
dstip: ['149.171.126.15', '149.171.126.14', '149.171.126.10', '149.171.126.13', '149.171.126.18', '149.171.126.19', '149.171.126.11', '149.171.126.17', '149.171.126.12', '149.171.126.16'] type: nominal
dsport: 786 type: integer
proto: 129 type: nominal
state: ['INT', 'CON', 'REQ', 'FIN', 'ACC', 'CLO'] type: nominal
dur: 8748 type: float
sbytes: 2604 type: integer
dbytes: 2410 type: integer
sttl: [254, 0, 62, 255, 63] type: integer
dttl: [0, 60, 252, 253] type: integer
sloss: 186 type: integer
dloss: 200 type: integer
service: ['-', 'dhcp', 'dns', 'http', 'ftp-data', 'smtp', 'pop3', 'ftp', 'snmp', 'radius', 'ssl', 'ssh', 'irc'] type: nominal
sload: 9501 type: float
dload: 8654 type: float
spkts: 225 type: integer
dpkts: 245 type: integer
swin: [0, 255] type: integer
dwin: [0, 255] type: integer
stcpb: 8584 type: integer
dtcpb: 8552 type: integer
smeansz: 1072 type: integer
dme

In [6]:
#what is the maximum port value?
#attack_sports = packet_data_df[packet_data_df['label'] == 1]['sport']
for k in range(0, packet_data_df['sport'].shape[0]):
    if (isinstance(packet_data_df.loc[k, 'sport'], str)):
        packet_data_df.loc[k, 'sport'] = int(packet_data_df.loc[k, 'sport'])
        
#did we convert all the strings?
for k in range(0, packet_data_df['sport'].shape[0]):
    if (isinstance(packet_data_df.loc[k, 'sport'], str)):
        print(packet_data_df.loc[k, 'sport'])
        
#print(packet_data_df.loc[k, 'sport'])
#sorted_ports = attack_sports.sort_values(ascending=False)

In [7]:
#How can we encode these various features, many of which are discrete integers?
#One-hot or Binary encoding seems logical, using Binary coding to keep things compact.

#Returns a list where each element are a 1 or 0, determining the binary encoding of value with
#at least bits number of bits. If the value cannot be encoding with the requested number of bits,
#None will be returned.
def binary_encode(value, bits):
    encoding = []
    while value != 0:
        encoding.append(value % 2)
        value //= 2
        
    if bits < len(encoding):
        return None #couldn't represent with requested number of bits
    
    while len(encoding) < bits:
        encoding.append(0)
    
    encoding.reverse()
    return encoding
        
print(binary_encode(7, 4)) #returns [0,1,1,1]
print(binary_encode(255, 2)) #returns None

[0, 1, 1, 1]
None


In [8]:
#try converting an ip into a bit array
ip_as_bits = []
for byte in packet_data_df['srcip'][0].split('.'):
    ip_as_bits += binary_encode(int(byte), 8)
    
print(ip_as_bits)

[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [9]:
#lets see what some of the other relevant fields look like
timestamp = packet_data_df['stime'][0]
print(timestamp)
nbits = 36
print(binary_encode(timestamp, nbits))
#can all the timestamps be represented with fewer bits?
for k in packet_data_df['stime']:
    if binary_encode(k, nbits) is None:
        print("Couldn't map all the timestamps!")
        break

1421927415
[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]


In [10]:
#what features do I care about?
#all the non-aggregate features that are some combination of the other features
#either directly in an example or temporal combinations, since these should ostensibly
#be discovered by the GAN
features_to_use = features_df[:39]
print(features_to_use[['name', 'type']])

                name       type
0              srcip    nominal
1              sport    integer
2              dstip    nominal
3             dsport    integer
4              proto    nominal
5              state    nominal
6                dur      float
7             sbytes    integer
8             dbytes    integer
9               sttl    integer
10              dttl    integer
11             sloss    integer
12             dloss    integer
13           service    nominal
14             sload      float
15             dload      float
16             spkts    integer
17             dpkts    integer
18              swin    integer
19              dwin    integer
20             stcpb    integer
21             dtcpb    integer
22           smeansz    integer
23           dmeansz    integer
24       trans_depth    integer
25       res_bdy_len    integer
26              sjit      float
27              djit      float
28             stime  timestamp
29             ltime  timestamp
30      

Feature encoding
-------------------------

I will encode the integer based features using a binary representation, using the minimum number of bits to represent the max value plus one bit. Float based parameters will be scaled in a typical manner.

IP addresses in particular are a special case, since each field is represending a collection of 4 bytes. These addresses will be represented as 32 bits, since this is the native representation and seems appropriate for this task.

In [11]:
def get_minimum_bits(value):
    min_bits = 1
    while binary_encode(value, min_bits) is None:
        min_bits += 1
        
    return min_bits

def build_input_feature_tensor(packet_data_dict):
    input_features = []
    
    srcip_segments = str(packet_data_dict['srcip']).split('.')
    srcip_bits = []
    for segment in srcip_segments:
        for k in binary_encode(int(segment), 8):
            srcip_bits.append(k)
    
    dstip_segments = str(packet_data_dict['dstip']).split('.')
    dstip_bits = []
    for segment in dstip_segments:
        for k in binary_encode(int(segment), 8):
            dstip_bits.append(k)
            
    sport = binary_encode(int(packet_data_dict['sport']), 16)#get_minimum_bits(int(packet_data_dict['sport'])) + 1)
    dport = binary_encode(int(packet_data_dict['dsport']), 16)#get_minimum_bits(int(packet_data_dict['dsport'])) + 1)
    
    #TODO need to encode the rest of the features buuuuuttttt that can come later.
    
    input_features += srcip_bits + dstip_bits + sport + dport
    
    return torch.tensor(input_features, dtype=torch.float32)
        
X = build_input_feature_tensor(packet_data_df.loc[0,:].to_dict())

#just playing with tensors here to figure out what I'm doing!
print(X)
print(X.shape)
X_seq = torch.tensor(()).new_zeros([5,1,X.shape[0]])
print(X_seq.shape)

#wow, I did not expect this to work!
X_seq[:,0,:] = X
#print(X_seq)

def build_input_sequence_tensor(packet_data_df, sequence_length):
    example_feature_vector = build_input_feature_tensor(packet_data_df.loc[0,:].to_dict())
    seq_out = torch.tensor(()).new_zeros([sequence_length, 1, example_feature_vector.shape[0]])
    
    for i in range(0, sequence_length):
        #print(seq_out.shape)
        seq_out[i,0,:] = build_input_feature_tensor(packet_data_df.loc[i,:].to_dict())
        
    return seq_out

X_seq = build_input_sequence_tensor(packet_data_df, 5)
print(X_seq)

tensor([0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
torch.Size([96])
torch.Size([5, 1, 96])
tensor([[[0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
          0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
          0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
  

In [15]:
#I shouldn't actually do the training in this notebook, this is mostly a test to see if I've prepared
#the features correctly for input to some RNN network.

#MODELS: Define Generator model and Discriminator model
#For the time being, this will be a one-layer RNN that is the same width as the input feature tensor

class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.hidden_size = hidden_size
        self.h0 = torch.zeros(1,1,hidden_size)
        self.gru = nn.GRU(input_size, hidden_size)
        self.f = f

    def forward(self, x):
        x, self.h0 = self.gru(x, self.h0)
        return self.f(x)
    
    def reset_hidden_state(self):
        self.h0 = torch.zeros(1,1,self.hidden_size)

#discriminator is copy of the generator
#class Discriminator(nn.Module):
#    def __init__(self, input_size, hidden_size, output_size, f):
#        super(Discriminator, self).__init__()
#        self.hidden_size = hidden_size
        #self.h0 = torch.zeros(1,1,hidden_size)
        #self.gru = nn.GRU(input_size, hidden_size)
#        self.map = nn.Linear(input_size, 1) # + hidden_size, 1)
#        self.f = f

#    def forward(self, x):
        #x, _ = self.gru(x, torch.zeros(1,1,self.hidden_size))
        #self.h0 = self.h0.detach() #need to do this so gradients don't back propogate infinitely in time
#        x = self.map(x) #torch.cat((x, self.h0),1))
#        return self.f(x)
    
#    def reset_hidden_state(self):
#        print("stub")
        #self.h0.detach_()
        #self.h0 = torch.zeros(1,1,self.hidden_size)
 

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f

    def forward(self, x):
        x = self.f(self.map1(x))
        x = self.f(self.map2(x))
        return self.f(self.map3(x))

In [16]:
#Define Generator and Discriminator
G = Generator(X_seq.shape[2], X_seq.shape[2], X_seq.shape[2], torch.sigmoid)
D = Discriminator(X_seq.shape[2], X_seq.shape[2], 1, torch.sigmoid)

#testing how resetting the hidden state has an effect on the output
G.zero_grad()
G.reset_hidden_state()
first = G(torch.tensor(X_seq).clone())
#print(first)

G.zero_grad()
G.reset_hidden_state()
second = G(torch.tensor(X_seq).clone())
#print(second)

#these outputs should be the same

/home/jaywalker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/jaywalker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [ ]:
import torch.optim as optim 

def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)  # Uniform-dist data into generator, _NOT_ Gaussian

def extract(v):
    return v.data.storage().tolist()

#well OK let's train the GAN on a single sequence and see what happens
def train(G, D, real_data):
    num_epochs = 25000
    print_interval = 1
    print_stats = True
    
    discriminator_learning_rate = 1e-3
    generator_learning_rate = 1e-3
    sgd_momentum = 0.9
    
    discriminator_training_steps = 1
    generator_training_steps = 1
    
    discriminator_fake_error, discriminator_real_error = 0.0, 0.0
    generator_error = 0
    
    criterion = nn.BCELoss() #right now the output is binary so this makes sense
    discriminator_optimizer = optim.SGD(D.parameters(), lr=discriminator_learning_rate, momentum=sgd_momentum)
    generator_optimizer = optim.SGD(G.parameters(), lr=discriminator_learning_rate, momentum=sgd_momentum)
    
    sequence_length = real_data.shape[0]
    for epoch in range(num_epochs):
        for discriminator_step in range(discriminator_training_steps):
            #D.reset_hidden_state()
            D.zero_grad()
            
            #Train D on the real samples
            #print(real_data.shape)
            discriminator_decision_r = D(real_data)
            #print(discriminator_decision_r.shape)
            discriminator_real_error = criterion(discriminator_decision_r, torch.ones(sequence_length))
            discriminator_real_error.backward()
            
            #Train D on the fake samples
            #D.reset_hidden_state()
            #D.zero_grad()
            
            #create a sample of length-1 real samples and generate the last
            generator_input = real_data[:sequence_length - 2, :, :]
            #print("Generator input shape: ", generator_input.shape)
            fake_data = torch.zeros(sequence_length - 1, 1, real_data.shape[2])
            fake_data[:sequence_length - 2, :, :] = generator_input
            
            fake_data_element = G(generator_input).detach()
            #print("Fake data element shape: ", fake_data_element.shape)
            #print("Fake data shape: ", fake_data.shape)
            fake_data[sequence_length - 2, :, :] = fake_data_element[2, :, :] #generate the last packet in the sequence
            discriminator_decision_f = D(fake_data)
            discriminator_fake_error = criterion(discriminator_decision_f, torch.zeros(sequence_length - 1))
            discriminator_fake_error.backward()
            discriminator_optimizer.step()
            
            dre = extract(discriminator_real_error)[0]
            dfe = extract(discriminator_fake_error)[0]
    
        for generator_step in range(generator_training_steps):
            print("generator step stub")
            
            
        if print_stats:
            if epoch % print_interval == 0 or epoch == num_epochs-1:
                print("D Real Error: ", dre)
                print("D Fake Error: ", dfe)
            
            
train(G,D,X_seq)

generator step stub
D Real Error:  0.6884706020355225
D Fake Error:  0.687537670135498
generator step stub
D Real Error:  0.6884700059890747
D Fake Error:  0.6875372529029846
generator step stub
D Real Error:  0.6884691715240479
D Fake Error:  0.687536358833313
generator step stub
D Real Error:  0.6884678602218628
D Fake Error:  0.6875351071357727
generator step stub
D Real Error:  0.6884661912918091
D Fake Error:  0.6875335574150085
generator step stub
D Real Error:  0.6884642839431763
D Fake Error:  0.687531590461731
generator step stub
D Real Error:  0.6884620189666748
D Fake Error:  0.6875295639038086
generator step stub
D Real Error:  0.6884596943855286
D Fake Error:  0.6875271797180176
generator step stub
D Real Error:  0.6884568333625793
D Fake Error:  0.6875245571136475
generator step stub
D Real Error:  0.6884539723396301
D Fake Error:  0.6875216960906982
generator step stub
D Real Error:  0.6884508728981018
D Fake Error:  0.6875187754631042
generator step stub
D Real Error:  

generator step stub
D Real Error:  0.6879278421401978
D Fake Error:  0.6870219111442566
generator step stub
D Real Error:  0.6879226565361023
D Fake Error:  0.687017023563385
generator step stub
D Real Error:  0.6879174709320068
D Fake Error:  0.6870120167732239
generator step stub
D Real Error:  0.687912106513977
D Fake Error:  0.6870071887969971
generator step stub
D Real Error:  0.6879069209098816
D Fake Error:  0.6870020627975464
generator step stub
D Real Error:  0.6879016757011414
D Fake Error:  0.6869972348213196
generator step stub
D Real Error:  0.6878964900970459
D Fake Error:  0.6869922280311584
generator step stub
D Real Error:  0.6878911256790161
D Fake Error:  0.6869872212409973
generator step stub
D Real Error:  0.6878859400749207
D Fake Error:  0.686982274055481
generator step stub
D Real Error:  0.6878805756568909
D Fake Error:  0.6869773864746094
generator step stub
D Real Error:  0.6878753900527954
D Fake Error:  0.6869723796844482
generator step stub
D Real Error:  

generator step stub
D Real Error:  0.6873579025268555
D Fake Error:  0.6864855885505676
generator step stub
D Real Error:  0.687352180480957
D Fake Error:  0.6864801645278931
generator step stub
D Real Error:  0.6873465180397034
D Fake Error:  0.6864747405052185
generator step stub
D Real Error:  0.6873406767845154
D Fake Error:  0.6864694952964783
generator step stub
D Real Error:  0.6873350143432617
D Fake Error:  0.6864640116691589
generator step stub
D Real Error:  0.6873291730880737
D Fake Error:  0.6864585876464844
generator step stub
D Real Error:  0.687323272228241
D Fake Error:  0.6864533424377441
generator step stub
D Real Error:  0.6873176097869873
D Fake Error:  0.6864478588104248
generator step stub
D Real Error:  0.6873117685317993
D Fake Error:  0.6864423751831055
generator step stub
D Real Error:  0.6873059868812561
D Fake Error:  0.6864370107650757
generator step stub
D Real Error:  0.6873001456260681
D Fake Error:  0.6864316463470459
generator step stub
D Real Error: 

generator step stub
D Real Error:  0.6867108345031738
D Fake Error:  0.6858832240104675
generator step stub
D Real Error:  0.6867045760154724
D Fake Error:  0.6858772039413452
generator step stub
D Real Error:  0.6866981983184814
D Fake Error:  0.6858713626861572
generator step stub
D Real Error:  0.6866917610168457
D Fake Error:  0.6858655214309692
generator step stub
D Real Error:  0.6866854429244995
D Fake Error:  0.6858595609664917
generator step stub
D Real Error:  0.6866791248321533
D Fake Error:  0.6858536005020142
generator step stub
D Real Error:  0.6866726875305176
D Fake Error:  0.6858476996421814
generator step stub
D Real Error:  0.6866663098335266
D Fake Error:  0.6858417987823486
generator step stub
D Real Error:  0.6866598129272461
D Fake Error:  0.6858359575271606
generator step stub
D Real Error:  0.6866533756256104
D Fake Error:  0.6858299970626831
generator step stub
D Real Error:  0.6866469979286194
D Fake Error:  0.6858240365982056
generator step stub
D Real Error

generator step stub
D Real Error:  0.6859018206596375
D Fake Error:  0.6851391792297363
generator step stub
D Real Error:  0.6858947277069092
D Fake Error:  0.685132622718811
generator step stub
D Real Error:  0.6858875155448914
D Fake Error:  0.6851260662078857
generator step stub
D Real Error:  0.6858804225921631
D Fake Error:  0.6851195096969604
generator step stub
D Real Error:  0.6858732104301453
D Fake Error:  0.6851129531860352
generator step stub
D Real Error:  0.6858659982681274
D Fake Error:  0.6851065158843994
generator step stub
D Real Error:  0.6858587861061096
D Fake Error:  0.6851000189781189
generator step stub
D Real Error:  0.6858516931533813
D Fake Error:  0.6850934028625488
generator step stub
D Real Error:  0.6858444213867188
D Fake Error:  0.6850868463516235
generator step stub
D Real Error:  0.6858372092247009
D Fake Error:  0.6850802898406982
generator step stub
D Real Error:  0.6858299970626831
D Fake Error:  0.6850736737251282
generator step stub
D Real Error:

...

D Real Error:  0.5658802390098572
D Fake Error:  0.6360375285148621
generator step stub
D Real Error:  0.5658773183822632
D Fake Error:  0.6360374689102173
generator step stub
D Real Error:  0.5658745765686035
D Fake Error:  0.6360371708869934
generator step stub
D Real Error:  0.5658718347549438
D Fake Error:  0.6360369324684143
generator step stub
D Real Error:  0.5658689737319946
D Fake Error:  0.6360368132591248
generator step stub
D Real Error:  0.5658661723136902
D Fake Error:  0.6360365152359009
generator step stub
D Real Error:  0.5658633708953857
D Fake Error:  0.6360362768173218
generator step stub
D Real Error:  0.5658606290817261
D Fake Error:  0.6360360980033875
generator step stub
D Real Error:  0.5658578276634216
D Fake Error:  0.6360358595848083
generator step stub
D Real Error:  0.565855085849762
D Fake Error:  0.6360356211662292
generator step stub
D Real Error:  0.5658522844314575
D Fake Error:  0.6360353827476501
generator step stub
D Real Error:  0.5658494234085083

generator step stub
D Real Error:  0.5655026435852051
D Fake Error:  0.6360076665878296
generator step stub
D Real Error:  0.5655000805854797
D Fake Error:  0.6360076069831848
generator step stub
D Real Error:  0.565497636795044
D Fake Error:  0.6360073089599609
generator step stub
D Real Error:  0.5654951333999634
D Fake Error:  0.6360071301460266
generator step stub
D Real Error:  0.5654925107955933
D Fake Error:  0.6360069513320923
generator step stub
D Real Error:  0.5654900670051575
D Fake Error:  0.6360067129135132
generator step stub
D Real Error:  0.5654875040054321
D Fake Error:  0.6360065340995789
generator step stub
D Real Error:  0.5654850602149963
D Fake Error:  0.6360062956809998
generator step stub
D Real Error:  0.5654824376106262
D Fake Error:  0.6360061764717102
generator step stub
D Real Error:  0.5654799342155457
D Fake Error:  0.6360059380531311
generator step stub
D Real Error:  0.5654774904251099
D Fake Error:  0.6360057592391968
generator step stub
D Real Error:

generator step stub
D Real Error:  0.5652422904968262
D Fake Error:  0.6359871029853821
generator step stub
D Real Error:  0.5652398467063904
D Fake Error:  0.6359870433807373
generator step stub
D Real Error:  0.5652375817298889
D Fake Error:  0.635986864566803
generator step stub
D Real Error:  0.5652351379394531
D Fake Error:  0.6359866857528687
generator step stub
D Real Error:  0.5652329325675964
D Fake Error:  0.635986328125
generator step stub
D Real Error:  0.5652306079864502
D Fake Error:  0.6359861493110657
generator step stub
D Real Error:  0.5652282238006592
D Fake Error:  0.6359860301017761
generator step stub
D Real Error:  0.5652258992195129
D Fake Error:  0.6359858512878418
generator step stub
D Real Error:  0.5652235746383667
D Fake Error:  0.6359856128692627
generator step stub
D Real Error:  0.5652211904525757
D Fake Error:  0.6359854936599731
generator step stub
D Real Error:  0.565218985080719
D Fake Error:  0.635985255241394
generator step stub
D Real Error:  0.56

generator step stub
D Real Error:  0.564987301826477
D Fake Error:  0.6359670162200928
generator step stub
D Real Error:  0.5649849772453308
D Fake Error:  0.6359668374061584
generator step stub
D Real Error:  0.5649828910827637
D Fake Error:  0.6359667778015137
generator step stub
D Real Error:  0.5649808049201965
D Fake Error:  0.6359664797782898
generator step stub
D Real Error:  0.5649785995483398
D Fake Error:  0.6359663605690002
generator step stub
D Real Error:  0.5649765729904175
D Fake Error:  0.6359660625457764
generator step stub
D Real Error:  0.5649741888046265
D Fake Error:  0.6359660029411316
generator step stub
D Real Error:  0.5649721026420593
D Fake Error:  0.6359658241271973
generator step stub
D Real Error:  0.5649698972702026
D Fake Error:  0.6359657049179077
generator step stub
D Real Error:  0.5649677515029907
D Fake Error:  0.6359654664993286
generator step stub
D Real Error:  0.5649656653404236
D Fake Error:  0.6359652280807495
generator step stub
D Real Error:

generator step stub
D Real Error:  0.5647128224372864
D Fake Error:  0.6359453797340393
generator step stub
D Real Error:  0.5647107362747192
D Fake Error:  0.6359453797340393
generator step stub
D Real Error:  0.5647088289260864
D Fake Error:  0.6359450817108154
generator step stub
D Real Error:  0.5647069215774536
D Fake Error:  0.6359449625015259
generator step stub
D Real Error:  0.5647048354148865
D Fake Error:  0.6359447836875916
generator step stub
D Real Error:  0.5647028088569641
D Fake Error:  0.6359446048736572
generator step stub
D Real Error:  0.5647009611129761
D Fake Error:  0.6359444260597229
generator step stub
D Real Error:  0.5646988749504089
D Fake Error:  0.6359443664550781
generator step stub
D Real Error:  0.5646968483924866
D Fake Error:  0.6359441876411438
generator step stub
D Real Error:  0.5646949410438538
D Fake Error:  0.6359440088272095
generator step stub
D Real Error:  0.5646928548812866
D Fake Error:  0.6359438896179199
generator step stub
D Real Error